# Homework 5

In [2]:
import pyspark
from pyspark.sql import SparkSession

## Question 1

In [3]:
pyspark.__version__

'3.4.2'

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/26 11:23:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# df = spark.read.parquet("yellow_tripdata_2024-10.parquet")
df = spark.read.parquet('data/')

In [12]:
df \
    .repartition(4) \
    .write.parquet('data/')

In [6]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-12 04:16:59|  2024-10-12 04:24:37|              2|         2.07|         1|                 N|         125|         100|           1|       10.7|  1.0|    0.5|       2.

## Question 3

In [9]:
df \
    .filter("DATE(tpep_pickup_datetime) = '2024-10-15' AND DATE(tpep_dropoff_datetime) = '2024-10-15'") \
    .selectExpr("count(1) AS count_trips") \
    .show()

+-----------+
|count_trips|
+-----------+
|     127993|
+-----------+



## Question 4
longest trip length in hours

In [64]:
df \
    .selectExpr("(unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS trip_duration") \
    .orderBy("trip_duration", ascending=False) \
    .limit(1) \
    .show()

+------------------+
|     trip_duration|
+------------------+
|162.61777777777777|
+------------------+



## Question 6

In [11]:
# !wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [6]:
df_zone = spark.read.csv('taxi_zone_lookup.csv', header=True)

In [7]:
df_zone.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [8]:
df_zone.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [12]:
df_join = df.join(df_zone, df.PULocationID == df_zone.LocationID, how='inner')

In [13]:
df_join.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee',
 'LocationID',
 'Borough',
 'Zone',
 'service_zone']

In [19]:
df_join.groupBy("Zone") \
    .count() \
    .withColumnRenamed("count", "trip_frequency") \
    .orderBy("trip_frequency", ascending=True) \
    .limit(1) \
    .show()

+--------------------+--------------+
|                Zone|trip_frequency|
+--------------------+--------------+
|Governor's Island...|             1|
+--------------------+--------------+

